<a href="https://colab.research.google.com/github/yogisaireddy14/Intellifailabs_assignment/blob/main/Intellifailabs_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/KwaiVGI/LivePortrait.git
%cd LivePortrait

In [ ]:
!pip install -q --upgrade pip
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121
!pip install -r requirements.txt

In [ ]:
!pip install transformers==4.41.0
!pip install numpy==1.24.4 onnxruntime-gpu==1.17.1

In [ ]:
import torch
print("PyTorch CUDA available? ", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Current device:", torch.cuda.get_device_name(0))
    print("CUDA capability:", torch.version.cuda)
else:
    print("No GPU detected.")

In [ ]:
!huggingface-cli download KwaiVGI/LivePortrait \
  --local-dir pretrained_weights \
  --exclude "*.git*" "README.md" "docs"

In [ ]:
import torch
_ = torch.zeros((1,1), device="cuda")
torch.cuda.synchronize()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi --query-gpu=memory.used --format=csv,nounits,noheader

In [ ]:
import time
import subprocess
import sys
SRC_IMG = "/content/LivePortrait/assets/examples/source/s9.jpg"
DRIVE_VID = "/content/LivePortrait/assets/examples/driving/d0.mp4"
OUT_DIR = "animations"

start_time = time.time()
try:
    result = subprocess.run(
        [
            sys.executable, "/content/LivePortrait/inference.py",
            "-s", SRC_IMG,
            "-d", DRIVE_VID,
            "-o", OUT_DIR
        ],
        check=True,
        capture_output=True,
        text=True
    )

except subprocess.CalledProcessError as e:
    print(" inference.py exited with a non-zero status.")
    print("------ stdout ------")
    print(e.stdout)
    print("------ stderr ------")
    print(e.stderr)
    raise

end_time = time.time()

elapsed_ms = (end_time - start_time) * 1000
print(f"=== Elapsed inference time (FP32): {elapsed_ms:.0f} ms total ===")


In [ ]:
import torch
from src.modules.spade_generator import SPADEDecoder

generator = SPADEDecoder()
generator.eval().cuda()

ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/base_models/spade_generator.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")

generator.load_state_dict(state_dict, strict=False)

dummy_input = torch.randn(512, 256, 3, 3).cuda()
with torch.cuda.amp.autocast(), torch.no_grad():
    output = generator(dummy_input)

print(" Inference complete. Output shape:", output.shape)

traced = torch.jit.trace(generator, dummy_input)
torchscript_path = "/content/spade_generator_torchscript.pt"
traced.save(torchscript_path)
print(f" TorchScript model saved at: {torchscript_path}")

In [ ]:
for _ in range(10):
    _ = generator(dummy_input)

# Time it
start = time.time()
for _ in range(100):
    _ = generator(dummy_input)
end = time.time()

print(f"Original Model Avg Inference Time: {(end - start)/100:.6f} sec")
optimized_model = torch.jit.load("/content/spade_generator_torchscript.pt").eval().cuda()

# Warm-up
for _ in range(10):
    _ = optimized_model(dummy_input)

# Time it
start = time.time()
for _ in range(100):
    _ = optimized_model(dummy_input)
end = time.time()

print(f"TorchScript Model Avg Inference Time: {(end - start)/100:.6f} sec")

In [ ]:
import torch
from src.modules.motion_extractor import MotionExtractor

generator = MotionExtractor(num_kp=21)
generator.eval().cuda()

ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/base_models/motion_extractor.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")
generator.load_state_dict(state_dict, strict=True)

dummy_input = torch.randn(1,3,256,256).cuda()

with torch.cuda.amp.autocast(), torch.no_grad():
    output = dict(generator(dummy_input))

print(" Inference complete. Output shape:", output)


traced = torch.jit.trace(generator, dummy_input, strict=False)
torchscript_path = "/content/motion_extractor_torchscript.pt"
traced.save(torchscript_path)
print(f" TorchScript model saved at: {torchscript_path}")


In [ ]:
for _ in range(10):
    _ = generator(dummy_input)

start = time.time()
for _ in range(100):
    _ = generator(dummy_input)
end = time.time()

print(f"Original Model Avg Inference Time: {(end - start)/100:.6f} sec")
optimized_model = torch.jit.load("/content/motion_extractor_torchscript.pt").eval().cuda()


for _ in range(10):
    _ = optimized_model(dummy_input)


start = time.time()
for _ in range(100):
    _ = optimized_model(dummy_input)
end = time.time()

print(f"TorchScript Model Avg Inference Time: {(end - start)/100:.6f} sec")

In [ ]:
import torch
from src.modules.appearance_feature_extractor import AppearanceFeatureExtractor


model = AppearanceFeatureExtractor(
    image_channel=3,
    block_expansion=64,
    num_down_blocks=2,
    max_features=512,
    reshape_channel=32,
    reshape_depth=16,
    num_resblocks=6
)
model.eval().cuda()


ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/base_models/appearance_feature_extractor.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")
model.load_state_dict(state_dict, strict=True)


dummy_input = torch.randn(1, 3, 256, 256).cuda()


traced = torch.jit.trace(model, dummy_input, strict=False)


torchscript_path = "/content/appearance_feature_extractor.pt"
traced.save(torchscript_path)

print(f" TorchScript model saved at: {torchscript_path}")


In [ ]:
for _ in range(10):
    _ = generator(dummy_input)


start = time.time()
for _ in range(100):
    _ = generator(dummy_input)
end = time.time()

print(f"Original Model Avg Inference Time: {(end - start)/100:.6f} sec")
optimized_model = torch.jit.load("/content/appearance_feature_extractor.pt").eval().cuda()


for _ in range(10):
    _ = optimized_model(dummy_input)


start = time.time()
for _ in range(100):
    _ = optimized_model(dummy_input)
end = time.time()

print(f"TorchScript Model Avg Inference Time: {(end - start)/100:.6f} sec")

In [ ]:
import torch
from src.modules.stitching_retargeting_network import StitchingRetargetingNetwork


generator = StitchingRetargetingNetwork( input_size=256,
    hidden_sizes=[512, 512, 256],
    output_size=3)
generator.eval().cuda()


ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/retargeting_models/stitching_retargeting_module.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")
for name, param in generator.named_parameters():
    print(name, param.shape)

generator.load_state_dict(state_dict, strict=False)


dummy_input = torch.randn(1, 256).cuda()
with torch.cuda.amp.autocast(), torch.no_grad():
    output = generator(dummy_input)

print(" Inference complete. Output shape:", output.shape)


traced = torch.jit.trace(generator, dummy_input)
torchscript_path = "/content/stitching_retargeting_torchscript.pt"
traced.save(torchscript_path)
print(f" TorchScript model saved at: {torchscript_path}")

In [ ]:
import torch
import time
from src.modules.stitching_retargeting_network import StitchingRetargetingNetwork


generator = StitchingRetargetingNetwork( input_size=256,
    hidden_sizes=[512, 512, 256],
    output_size=3)
ckpt_path = "/content/LivePortrait/pretrained_weights/liveportrait/retargeting_models/stitching_retargeting_module.pth"
state_dict = torch.load(ckpt_path, map_location="cuda")
generator.load_state_dict(state_dict, strict=False)
generator.eval().cuda()
dummy_input = torch.randn(1, 256).cuda()
for _ in range(10):
    _ = generator(dummy_input)


start = time.time()
for _ in range(100):
    _ = generator(dummy_input)
end = time.time()

print(f"Original Model Avg Inference Time: {(end - start)/100:.6f} sec")
optimized_model = torch.jit.load("/content/stitching_retargeting_torchscript.pt").eval().cuda()


for _ in range(10):
    _ = optimized_model(dummy_input)

start = time.time()
for _ in range(100):
    _ = optimized_model(dummy_input)
end = time.time()

print(f"TorchScript Model Avg Inference Time: {(end - start)/100:.6f} sec")

In [ ]:
import time
import os
import torch
from src.config.inference_config import InferenceConfig
from src.config.crop_config import CropConfig
from src.live_portrait_pipeline import LivePortraitPipeline
from IPython.display import Video, display
from src.config.argument_config import ArgumentConfig

SRC_IMG = "/content/LivePortrait/assets/examples/source/s9.jpg"
DRIVE_VID = "/content/LivePortrait/assets/examples/driving/d0.mp4"
OUT_DIR = "animations"
os.makedirs(OUT_DIR, exist_ok=True)


parsed_args = ArgumentConfig(source=SRC_IMG, driving=DRIVE_VID, output_dir=OUT_DIR)

pipeline = LivePortraitPipeline(inference_cfg=InferenceConfig(), crop_cfg=CropConfig())

start_time = time.time()

with torch.cuda.amp.autocast(), torch.no_grad():
    animation = pipeline.execute(parsed_args)


end_time = time.time()

elapsed_ms = (end_time - start_time) * 1000
print(f"=== Elapsed inference time (AMP only): {elapsed_ms:.0f} ms total ===")
